In [ ]:
import numpy as np
from tabulate import tabulate
import random
import heapq
import sys

# Emojis/symbols representing each element on the ship
wallSymbol = str('\U00002B1B')
floorSymbol = str('\U00002B1C')
botSymbol = str('\U0001F916')
buttonSymbol = str('\U0001F7E5')
fireSymbol = str('\U0001F525')

D = 40  # Ship dimensions (D X D)
q = 0.7 # Percent variable to affect ship flammability ()
pv = 0.2 # Percent variable to affect ship openness (0 = default ship, 0.5 = moderately open, 1 = completely open)

# Function to generate the ship layout
def theShip(D, opencells):
    options = [] # List of possible closed cells to open
    ship = np.full((D, D), int(0), dtype=object) # Initiate the ship (D X D)
    X, Y = random.randrange(D), random.randrange(D)  # Select a random point on the ship
    opencells.append(tuple([X, Y]))
    markOpen(ship, X, Y, D, options) # opencells is the list of all cells that have been opened, MarkOpen function initiates cell opening process

    while len(options) >= 1: #pull from list of possible closed cells and stop after it empties, choose one at random each iteration to open using MarkOpen
        rand = random.randrange(len(options))
        X, Y = options[rand]
        opencells.append(tuple([X, Y]))
        options = markOpen(ship, X, Y, D, options)

    # Detect and handle dead ends
    deadends = [] # List of all open cells that is a dead end, only one neighboring open cell
    for X in range(D):
        for Y in range(D):
            if isinstance(ship[X][Y], int):
                ship[X][Y] = wallSymbol
            if isinstance(ship[X][Y], str):
                isDeadend = 0
                if X - 1 >= 0:
                    if ship[X - 1][Y] == floorSymbol:
                        isDeadend += 1
                if X + 1 < D:
                    if ship[X + 1][Y] == floorSymbol:
                        isDeadend += 1
                if Y - 1 >= 0:
                    if ship[X][Y - 1] == floorSymbol:
                        isDeadend += 1
                if Y + 1 < D:
                    if ship[X][Y + 1] == floorSymbol:
                        isDeadend += 1
                if isDeadend == 1:
                    deadends.append(tuple([X, Y]))
    # The above section checks each cell in the ship, determining whether it is a dead end by examining all neighbors

    random.shuffle(deadends)
    # Randomly shuffle the list of dead end open cells and divide them by 2. This half of the dead ends will be turned into non-deadend open cells
    for i in range(int(len(deadends)/2)):
        X, Y = deadends[i]
        randselect = []
        randnum = 0
        if Y - 1 >= 0:
            if isinstance(ship[X][Y - 1], int):
                randselect.append(ship[X][Y - 1])
                randnum += 1
        if X - 1 >= 0:
            if isinstance(ship[X - 1][Y], int):
                randselect.append(ship[X - 1][Y])
                randnum += 1
        if X + 1 < D:
            if isinstance(ship[X + 1][Y], int):
                randselect.append(ship[X + 1][Y])
                randnum += 1
        if Y + 1 < D:
            if isinstance(ship[X][Y + 1], int):
                randselect.append(ship[X][Y + 1])
                randnum += 1
        # Above section figures out which cells around it are closed so that it can randomly select one

        if randnum > 0:
            randnum = random.randrange(randnum)
            deadends.remove(tuple([X, Y]))
            if Y - 1 >= 0 and isinstance(ship[X][Y - 1], int):
                if randnum == 0:
                    ship[X][Y - 1] = str(floorSymbol)
                    opencells.append(tuple([X, Y - 1]))
                randnum -= 1
            if X - 1 >= 0 and isinstance(ship[X - 1][Y], int):
                if randnum == 0:
                    ship[X - 1][Y] = str(floorSymbol)
                    opencells.append(tuple([X - 1, Y]))
                randnum -= 1
            if X + 1 < D and isinstance(ship[X + 1][Y], int):
                if randnum == 0:
                    ship[X + 1][Y] = str(floorSymbol)
                    opencells.append(tuple([X + 1, Y]))
                randnum -= 1
            if Y + 1 < D and isinstance(ship[X][Y + 1], int):
                if randnum == 0:
                    ship[X][Y + 1] = str(floorSymbol)
                    opencells.append(tuple([X, Y + 1]))
        # Above section chooses a random number in the range of 0 to number of available closed neighbors, essentially selecting one at random and opening

    for X in range(D):
        for Y in range(D):
            if ship[X][Y] == wallSymbol:
                if random.uniform(0, 1) <= pv:
                    ship[X][Y] = str(floorSymbol)
            if ship[X][Y] == floorSymbol and (tuple([X, Y])) not in opencells: 
                opencells.append(tuple([X, Y]))
    # The above section is PURELY for the bonus question. It has a chance (p) of opening every closed cell it passes, used the ship way more open. 
    return opencells, ship


# Function to open the cell and add its neighbors to the options list
def markOpen(ship, X, Y, D, options):
    ship[X][Y] = str(floorSymbol) # Essential function to open the cell / make it a floor

    if (X, Y) in options:
        options.remove(tuple([X, Y])) # Remove a cell from options after opening, meaning it is no longer a cell that is available to open

    if Y - 1 >= 0 and isinstance(ship[X][Y - 1], int):
        ship[X][Y - 1] += 1
        if ship[X][Y - 1] == 1:
            options.append(tuple([X, Y - 1]))
        elif ship[X][Y - 1] > 1 and (X, Y - 1) in options:
            options.remove(tuple([X, Y - 1]))
    if X - 1 >= 0 and isinstance(ship[X - 1][Y], int):
        ship[X - 1][Y] += 1
        if ship[X - 1][Y] == 1:
            options.append(tuple([X - 1, Y]))
        elif ship[X - 1][Y] > 1 and (X - 1, Y) in options:
            options.remove(tuple([X - 1, Y]))
    if X + 1 < D and isinstance(ship[X + 1][Y], int):
        ship[X + 1][Y] += 1
        if ship[X + 1][Y] == 1:
            options.append(tuple([X + 1, Y]))
        elif ship[X + 1][Y] > 1 and (X + 1, Y) in options:
            options.remove(tuple([X + 1, Y]))
    if Y + 1 < D and isinstance(ship[X][Y + 1], int):
        ship[X][Y + 1] += 1
        if ship[X][Y + 1] == 1:
            options.append(tuple([X, Y + 1]))
        elif ship[X][Y + 1] > 1 and (X, Y + 1) in options:
            options.remove(tuple([X, Y + 1]))
    """The above function ensures that the options for cells to be opened are updated every time a cell is opened. 
    It kind of operates like Minesweeper, where the number indicates a closed cell and how many open cells surrounding it.
    A ship[x][y] value of 1 means it has 1 open cell neighbor and IS a suitable option to opem
    A ship[x][y] value of 0 means it has no open cell neighbors and IS NOT a suitable option to open
    A ship[x][y] value of 2 or higher means it has 2 or more open cells neighbors and IS NOT a suitable option to open"""
    return options


# Place the bot, button, and fire
def placeObjects(bot, button, fire0, ship, opencells, fireNeighbors):
    ship[bot[0]][bot[1]] = botSymbol # Bots 1 through 4 start at this location on the ship
    ship[button[0]][button[1]] = buttonSymbol # The button starts at this spot
    fireNeighbors.append(tuple([fire0[0], fire0[1]])) # The fire starts here
    fireNeighbors, opencells = startFire(fire0, ship, opencells, fireNeighbors) # startFire is the function that actually places fire or spreads fire at any given point
    print(tabulate(ship))
    return fireNeighbors, opencells

# Fire spread logic
def spreadFire(fireNeighbors, opencells, ship):
    toFire = [] # This is the list of open cells that will be ignited this turn

    for i in fireNeighbors: # The next 9 lines determines K in the formula
        K = 0
        if i[0] - 1 >= 0 and ship[i[0] - 1][i[1]] == fireSymbol:
            K += 1
        if i[0] + 1 < D and ship[i[0] + 1][i[1]] == fireSymbol:
            K += 1
        if i[1] - 1 >= 0 and ship[i[0]][i[1] - 1] == fireSymbol:
            K += 1
        if i[1] + 1 < D and ship[i[0]][i[1] + 1] == fireSymbol:
            K += 1 
        p = (1 - ((1 - q) ** K)) # Give procedural that determines if a fire will start at a new cell, using given q and calculated K
        if random.uniform(0, 1) <= p: # If random value is lower than p, this cell will be ignited / added to toFire
            toFire.append(tuple(i)) 

    for i in toFire:
        startFire(i, ship, opencells, fireNeighbors) # Initiate function to start fires at each toFire cell

    return opencells, fireNeighbors


# Logic to turn open cells / button cells / bot cells into fire cells
def startFire(Fire, ship, opencells, fireNeighbors):
    ship[Fire[0]][Fire[1]] = fireSymbol
    fireNeighbors.remove(tuple([Fire[0], Fire[1]])) # Remove the new fire cell from list containing neighbors of fire
    if (tuple([Fire[0], Fire[1]])) in opencells:
        opencells.remove(tuple([Fire[0], Fire[1]])) # Remove the new fire cell from list containing open cells
    if Fire[1] - 1 >= 0 and (ship[Fire[0]][Fire[1] - 1] in [floorSymbol, botSymbol, buttonSymbol]):
        fireNeighbors.append(tuple([Fire[0], Fire[1] - 1]))
    if Fire[1] + 1 < D and (ship[Fire[0]][Fire[1] + 1] in [floorSymbol, botSymbol, buttonSymbol]):
        fireNeighbors.append(tuple([Fire[0], Fire[1] + 1]))
    if Fire[0] + 1 < D and (ship[Fire[0] + 1][Fire[1]] in [floorSymbol, botSymbol, buttonSymbol]):
        fireNeighbors.append(tuple([Fire[0] + 1, Fire[1]]))
    if Fire[0] - 1 >= 0 and (ship[Fire[0] - 1][Fire[1]] in [floorSymbol, botSymbol, buttonSymbol]):
        fireNeighbors.append(tuple([Fire[0] - 1, Fire[1]]))
    return fireNeighbors, opencells # Adds cells neighboring new fire cell to list containing neighbors of fire


# Heuristic function for A*
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


# Get neighbors and avoid fire and walls
def get_neighbors(ship, node, fire, fireNeighbors, bot_choice):
    neighbors = []
    x, y = node
    potential_moves = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
    if bot_choice != 3:
        for nx, ny in potential_moves:
            if 0 <= nx < D and 0 <= ny < D and ship[nx][ny] != wallSymbol and ship[nx][ny] != fireSymbol: # All bots don't consider walls or fires
                neighbors.append((nx, ny))
    if bot_choice == 3:
        for nx, ny in potential_moves:
            if 0 <= nx < D and 0 <= ny < D and ship[nx][ny] != wallSymbol and ship[nx][ny] != fireSymbol: # Bot 3 won't consider walls or fires or neighbors of fires
                valid = 1
                if nx + 1 < D and ship[nx + 1][ny] in fireSymbol:
                    valid = 0
                if nx > 0 and ship[nx - 1][ny] in fireSymbol:
                    valid = 0
                if ny + 1 < D and ship[nx][ny + 1] in fireSymbol:
                    valid = 0
                if ny > 0 and ship[nx][ny - 1] in fireSymbol:
                    valid = 0
                if valid == 1:
                    neighbors.append((nx, ny))
    return neighbors


# A* algorithm for the bot
def a_star(ship, start, goal, fire, fireNeighbors, bot_choice):
    close_set = set()
    came_from = {}
    gscore = {start: 0}
    fscore = {start: heuristic(start, goal)}

    open_heap = []
    heapq.heappush(open_heap, (fscore[start], start))

    while open_heap:
        current = heapq.heappop(open_heap)[1]

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path = path[::-1]
            return path

        close_set.add(current)

        for neighbor in get_neighbors(ship, current, fire, fireNeighbors, bot_choice):
            # Calculate the tentative g score using the 'edge weight'. It is 1 for all cells if it is Bots 1, 2, or 3
            if bot_choice == 4:
                fireFactor = 1
                for i in range(-int(q*10), int(q*10)):
                    for j in range(-int(q*10), int(q*10)): # Bot 4 square scanner parameters, this is how it determines the 'edge weight' associated with traveling to a cell
                        if 0 < (neighbor[0] + i) < D and 0 < (neighbor[1] + j) < D:
                            if ship[neighbor[0] + i][neighbor[1] + j] == fireSymbol:
                                fireFactor *= 2
                tentative_gscore = gscore[current] + fireFactor # Bot 4 travel cost
            else:
                tentative_gscore = gscore[current] + 1 # Bots 1-3 travel cost

            if neighbor in close_set and tentative_gscore >= gscore.get(neighbor, 0):
                continue

            if tentative_gscore < gscore.get(neighbor, float('inf')):
                came_from[neighbor] = current
                gscore[neighbor] = tentative_gscore
                fscore[neighbor] = tentative_gscore + heuristic(neighbor, goal)
                heapq.heappush(open_heap, (fscore[neighbor], neighbor))

    if bot_choice == 3 or bot_choice == 4:
        bot_choice = 2
        return a_star(ship, start, goal, fire, fireNeighbors, bot_choice) # This is the functionality that allows Bot 3 and 4 to use Bot 2 path-finding if their methods don't work
    else:
        return False  # No path found


# Move the bot along the path
def bot_move(ship, bot, path, button, ti, death):
    if path:
        next_move = path.pop(0) # Next spot in path
        if ship[bot[0]][bot[1]] == fireSymbol:
            print("Can't move")
            print(ti)
            death = -1 # Kill the bot if it is in fire
        else:
            ship[bot[0]][bot[1]] = floorSymbol  # Clear bot's previous position
        bot[0], bot[1] = next_move
        if ship[bot[0]][bot[1]] == fireSymbol:
            print("Can't move")
            print(ti)
            death = -1 # Kill the bot if it is in fire
        else:
            ship[bot[0]][bot[1]] = botSymbol # Move the bot to next spot in path
    if tuple(bot) == button:
        print("The bot reached the button!") # Success
    return bot, path, death


# Run either Bot 1 or Bot 2
def run_bot(ship, bot, button, fire0, opencells, fireNeighbors):
    t = 150 # If 150 steps occur, the program will automatically stop. Typically no round will last more than ~50 time stamps
    ti = 0
    d1 = 0
    d2 = 0
    d3 = 0
    d4 = 0 # Booleans for whether the bot is dead or alive
    bot1 = bot.copy()
    bot2 = bot.copy()
    bot3 = bot.copy()
    bot4 = bot.copy() # Places all bots at the bot location

    # Calculate Bot 1's path once at the start
    bot_choice = 1
    path1 = a_star(ship, tuple(bot1), button, fire0, fireNeighbors, bot_choice)

    if not path1:
        print("No path found for Bot 1!")
        d1 = -1

    while ti < t and (d2 == 0 or d3 == 0 or d4 == 0): # This is the section that runs the bots and the fire spread every time step
        ti += 1
        print("Time:", ti)
        # Move Bot 1 using the pre-calculated path, does not call a* every time step like Bots 2, 3, 4
        bot_choice = 1
        if path1 and d1 == 0:
            print(bot_choice, "Bot - Shortest Path:", path1)
            print("Path Length:", len(path1))
            bot1, path1, d1 = bot_move(ship, bot1, path1, button, ti, d1) # Move bot
            if not path1: # Kill bot
                print("Bot 1 failed")
                d1 = -1

        # Recalculate and move Bot 2 at each time step
        bot_choice = 2
        path2 = a_star(ship, tuple(bot2), button, fire0, fireNeighbors, bot_choice) # Recalculate new path every time step
        if path2 and d2 == 0:
            print(bot_choice, "Bot - Shortest Path:", path2)
            print("Path Length:", len(path2))
            bot2, path2, d2 = bot_move(ship, bot2, path2, button, ti, d2) # Move bot
            if not path2: # Kill bot
                print("Bot 2 failed")
                d2 = -1
        else: # If path does not exist, kill bot
            d2 = -1
            print("No path found for Bot 2!")

        # Recalculate and move Bot 3 at each time step
        bot_choice = 3
        path3 = a_star(ship, tuple(bot3), button, fire0, fireNeighbors, bot_choice) # Recalculate new path every time step
        if path3 and d3 == 0:
            print(bot_choice, "Bot - Shortest Path:", path3)
            print("Path Length:", len(path3))
            bot3, path3, d3 = bot_move(ship, bot3, path3, button, ti, d3) # Move bot
            if not path3: # Kill bot
                print("Bot 3 failed")
                d3 = -1
        else: # If path does not exist, kill bot
            d3 = -1
            print("No path found for Bot 3!")

        # Recalculate and move Bot 4 at each time step
        bot_choice = 4
        path4 = a_star(ship, tuple(bot4), button, fire0, fireNeighbors, bot_choice) # Recalculate new path every time step
        if path4 and d4 == 0:
            print(bot_choice, "Bot - Shortest Path:", path4)
            print("Path Length:", len(path4))
            bot4, path4, d4 = bot_move(ship, bot4, path4, button, ti, d4) # Move bot
            if not path4: # Kill bot
                print("Bot 4 failed")
                d4 = -1
        else: # If path does not exist, kill bot
            d4 = -1
            print("No path found for Bot 4!")

        # Spread fire at each time step
        opencells, fireNeighbors = spreadFire(fireNeighbors, opencells, ship)
        print(tabulate(ship))

        # Check if any bot reached the button
        if tuple(bot1) == button:
            print("Bot 1 reached the button!")
            d1 = -1
        if tuple(bot2) == button:
            print("Bot 2 reached the button!")
            d2 = -1
        if tuple(bot3) == button:
            print("Bot 3 reached the button!")
            d3 = -1
        if tuple(bot4) == button:
            print("Bot 4 reached the button!")
            d4 = -1
       
def main():
    opencells = [] # List of cells that are open
    opencells, ship = theShip(D, opencells) # Initiate ship construction
    random.shuffle(opencells) # Randomize open cells (in order to randomly place button, bot, and fire)

    fireNeighbors = []
    bot = list(opencells[0])  # Bot's starting position
    button = opencells[1]     # Button's position
    fire0 = opencells[2]      # Initial fire's position

    placeObjects(bot, button, fire0, ship, opencells, fireNeighbors) # Initialize objects

    run_bot(ship, bot, button, fire0, opencells, fireNeighbors) # Begin simulation

if __name__ == "__main__":
    main()
